# Transcribe a YouTube Video in English with Assembly AI and yt-dlp

### References:

- [Assembly AI documentation](https://www.assemblyai.com/docs)
- [yt-dlp on GitHub](https://github.com/yt-dlp/yt-dlp)

## Preparation

#### Imports and Globals

In [1]:
import assemblyai as aai
import yt_dlp
import json

from config import *

aai.settings.api_key = aai_key
YT_BASE_URL = 'https://www.youtube.com/watch?v='
DST_FOLDER = 'files'

#### Task-specific Variables

In [2]:
# title = 'The Power To OverRule G-d'  # actually not needed as yt-dlp can obtain it from the video
v_id = 'wKS2sFRI87w'  # the main identifier of the video, absolutely needeed

#### Pull and save the soundtrack with yt-dlp

In [24]:
url = f'{YT_BASE_URL}{v_id}'

ydl_opts = {
    'format': 'm4a/bestaudio/best',  # The best audio version in m4a format
    'outtmpl': f'{DST_FOLDER}/%(title)s_%(id)s.%(ext)s',  
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(url)
    audio_file = ydl.prepare_filename(info)

print(f'\n>>> Downloaded to: {audio_file}')

[youtube] Extracting URL: https://www.youtube.com/watch?v=wKS2sFRI87w
[youtube] wKS2sFRI87w: Downloading webpage
[youtube] wKS2sFRI87w: Downloading tv client config
[youtube] wKS2sFRI87w: Downloading player c548b3da
[youtube] wKS2sFRI87w: Downloading tv player API JSON
[youtube] wKS2sFRI87w: Downloading ios player API JSON
[youtube] wKS2sFRI87w: Downloading m3u8 information
[info] wKS2sFRI87w: Downloading 1 format(s): 140
[download] files\The Power To OverRule G-d：_wKS2sFRI87w.m4a has already been downloaded
[download] 100% of   23.92MiB

>>> Downloaded to: files\The Power To OverRule G-d：_wKS2sFRI87w.m4a


## Building the transcript via AssemblyAI API

In [ ]:
config = aai.TranscriptionConfig(language_detection=True, speaker_labels=True)

In [26]:
transcriber = aai.Transcriber()
transcript = transcriber.transcribe(audio_file, config)
print(transcript.status)

TranscriptError: Failed to upload audio file: 
Reason: <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>

Request: <Request('POST', 'https://api.assemblyai.com/v2/upload')>

In [20]:
file_name = audio_file.split("\\")[-1].split(".")[0]
file_name

'The Power To OverRule G-d：_wKS2sFRI87w'

In [21]:
json.dump(transcript.json_response, 
          open(f'files/transcript_{file_name}_{transcript.id}.json', 'w', encoding='utf-8'), 
          indent=4, 
          ensure_ascii=False
)

with open(f'files/transcript_{file_name}_{transcript.id}.txt', 'w', encoding='utf-8') as f:
    f.write(transcript.text)